In [ ]:
# Import packages
import pandas as pd
from os import environ
from auth import Auth
from workspace import Workspace

# Tenant/app settings
TENANT_ID = environ.get('TENANT_ID', '')
CLIENT_ID = environ.get('CLIENT_ID', '')
CLIENT_SECRET = environ.get('CLIENT_SECRET', '')

In [ ]:
# Authentication (get bearer token)
auth = Auth(TENANT_ID, CLIENT_ID, CLIENT_SECRET)
token = auth.get_token()

In [ ]:
# Initializing object workspace
workspace = Workspace(token)

# Term to search on workspace name
workspace_to_search = 'Brewdat'

# Filter workspaces that contain search word or is Dataflows workspace
# Saves to an Excel (.xslx) file
workspaces = workspace.list_workspace(filters=f"contains(name,'{workspace_to_search}')%20or%20name%20eq%20'Dataflows'")
workspaces_list = workspaces.get('content', [])

# See content
pd.DataFrame(workspaces_list).head(3)

In [ ]:
responses = []

user = 'user@domain.com'

for data in workspaces_list:
    id = data.get('id', '')
    name = data.get('name', '')
    response = workspace.update_user(user_principal_name=user, workspace_id=id, access_right='Admin')

    # Try to update the user
    try:
        responses.append((id, name, 'Error', response['message']['content']))
    except:
        responses.append((id, name, 'Success', ''))

In [ ]:
# Create a dataframe with responses
df1 = pd.DataFrame(responses, columns=['id', 'name', 'status', 'error_message'])

# Serialize json from error message as a new dataframe
df2 = pd.json_normalize(df1['error_message'])

# Drop error message column and merge both dataframes
df1.drop(labels='error_message', axis='columns', inplace=True)
df = pd.merge(left=df1, right=df2, left_index=True, right_index=True)
df = df.fillna('')

# Save to an Excel file with user name
df.to_excel(f"./data/workspaces_{user.split('@')[0]}.xlsx", index=False)

In [ ]:
# Success
df.loc[df.status == 'Success']

In [ ]:
# Errors
df.loc[df.status != 'Success']